In [1]:
import numpy as np
from polyphys.manage.parser import TwoMonDepCub

import os

In [5]:
dddict = {
            "artifact": "am5.0nm2ac1.0nc100hl10.0sd2.0dt0.01bdump50adump100"
            "tdump500",
            "lineage": "ensemble_long",
            "group": "bug"
        }
two_mon = TwoMonDepCub(**dddict, ispath=False)
two_mon.adump

100

In [2]:
for artifact, lineage in [
    ('', 'segment'),
                ('am5.0nm2ac1.0nc100hl10.0sd2.0dt0.01bdump50adump100tdump500ens3.j01.bug', 'segment'),
                 ('am5.0nm2ac1.0nc100hl10.0sd2.0dt0.01bdump50adump100tdump500ens3.bug', 'whole'),
                 ('am5.0nm2ac1.0nc100hl10.0sd2.0dt0.01bdump50adump100tdump500', 'ensemble_long'),
                 ('nm2am5.0ac1.0nc100sd2.0', 'ensemble'),
                 ('nm2am5.0ac1.0nc100', 'space')
                ]:

    two_mon = TwoMonDepCub('./'+artifact+'.txt', lineage, 'bug', ispath=False)
    print(os.path.exists('./'+artifact+'.txt'))
    print(two_mon.filename)
    print(two_mon.filepath)
    print(two_mon.name)
    print()


'am' attribute not found in './.txt'
'nm' attribute not found in './.txt'
'ac' attribute not found in './.txt'
'nc' attribute not found in './.txt'
'hl' attribute not found in './.txt'
'sd' attribute not found in './.txt'
'dt' attribute not found in './.txt'
'bdump' attribute not found in './.txt'
'adump' attribute not found in './.txt'
'tdump' attribute not found in './.txt'
'ens' attribute not found in './.txt'
'j' attribute not found in './.txt'


AttributeError: 'TwoMonDepCub' object has no attribute 'dmon'

In [ ]:
two_mon = TwoMonDepCub('am1nm100ac1nc50000hl45sd0.1dt0.005bdump1000adump5000tdump5000ens1.j01.bug.txt','whole', 'bug')
print(two_mon.filename)
print(two_mon.filepath)
print(two_mon.name)

In [ ]:
two_mon = TwoMonDepCub('am1nm100ac1nc50000hl45sd0.1dt0.005bdump1000adump5000tdump5000ens1.bug.txt','whole', 'bug')

print(two_mon.filename)
print(two_mon.filepath)
print(two_mon.name)

assert os.path.dirname(os.path.abspath(two_mon.filename)) == two_mon.filepath

In [ ]:
two_mon = TwoMonDepCub('/home/research_data/am1nm100ac1nc50000hl45sd0.1dt0.005bdump1000adump5000tdump5000ens1.bug.txt','whole', 'bug')
print(two_mon.filename)
print(two_mon.filepath)
print(two_mon.name)


In [ ]:
two_mon = TwoMonDepCub('am1nm100ac1nc50000hl45sd0.1dt0.005bdump1000adump5000tdump5000ens1.bug','whole', 'bug')

print(two_mon.filename)
print(two_mon.filepath)
print(two_mon.name)



In [ ]:
def number_density_cube(
    n_atom: float,
    d_atom: float,
    l_cube: float,
    pbc: bool = False
) -> float:
    """
    Compute the bulk number density of a species in a cubic box.

    Parameters
    ----------
    n_atom : float
        Number of particles.
    d_atom : float
        Diameter of the particle of the species.
    l_cube : float
        Length of one side of the cubic box.
    pbc : bool
        Periodic boundary conditions along all box sides. If `True`,
        :math:`v_{avail} = l_{cube}^3`; otherwise,
        :math:`v_{avail} = (l_{cube} - d_{atom})^3`. Defaults to `False`.

    Returns
    -------
    float
        Bulk number density of the species in the cubic box.

    Notes
    -----
    The bulk number density is calculated as :math:`n_{atom} / v_{avail}`,
    where `v_{avail}` is the available volume to the center of geometry of a
    particle based on the presence of periodic boundary conditions.

    Examples
    --------
    >>> number_density_cube(1000, 1.0, 10.0)
    1.3717421124828531
    >>> number_density_cube(1000, 1.0, 10.0, pbc=True)
    1.0
    """
    v_avail = (l_cube - int(not pbc) * d_atom) ** 3
    return n_atom / v_avail


def volume_fraction_cube(
    n_atom: float,
    d_atom: float,
    l_cube: float,
    pbc: bool = False
) -> float:
    """
    Compute the volume fraction of a species in a cubic box.

    Parameters
    ----------
    n_atom : float
        Number of particles.
    d_atom : float
        Diameter of the particle of the species.
    l_cube : float
        Length of one side of the cubic box.
    pbc : bool
        Periodic boundary conditions along all box sides. If `True`,
        :math:`v_{avail} = l_{cube}^3`; otherwise,
        :math:`v_{avail} = (l_{cube} - d_{atom})^3`. Defaults to `False`.

    Returns
    -------
    float
        Volume fraction of the species in the cubic box.

    Notes
    -----
    The volume fraction is computed in the volume available to the center of
    geometry of each particle. For point-like particles, the notion of volume
    fraction is meaningless. For finite-size particles, the available volume
    depends on whether periodic boundary conditions (PBCs) are applied.

    Examples
    --------
    >>> volume_fraction_cube(1000, 1.0, 10.0)
    0.7189611722461486
    """
    rho = number_density_cube(n_atom, d_atom, l_cube, pbc)
    return rho * np.pi * d_atom ** 3 / 6


def number_density_cylinder(
    n_atom: float,
    d_atom: float,
    l_cyl: float,
    d_cyl: float,
    pbc: bool = False
) -> float:
    """
    Compute the bulk number density of a species in a cylindrical confinement.

    Parameters
    ----------
    n_atom : float
        Number of particles.
    d_atom : float
        Diameter of the particle of the species.
    l_cyl : float
        Length of the cylindrical confinement.
    d_cyl : float
        Diameter of the cylindrical confinement.
    pbc : bool
        Periodic boundary conditions along the longitudinal axis. If `True`,
        :math:`v_{avail} = \\pi * l_{cyl} * (d_{cyl} - d_{atom})^2 / 4`;
        otherwise, :math:`v_{avail} = \\pi * (l_{cyl} - d_{atom}) * (d_{cyl}
        - d_{atom})^2 / 4`. Defaults to `False`.

    Returns
    -------
    float
        Bulk number density of the species in the cylindrical confinement.

    Notes
    -----
    The bulk number density is calculated as :math:`n_{atom} / v_{avail}`,
    where `v_{avail}` is the available volume to the center of geometry of a
    particle based on the presence of periodic boundary conditions.

    Examples
    --------
    >>> number_density_cylinder(1000, 1.0, 10.0, 5.0)
    13.592168292790723
    """
    v_avail = np.pi * (l_cyl - int(not pbc) * d_atom) * (d_cyl - d_atom) ** 2 / 4
    return n_atom / v_avail


def volume_fraction_cylinder(
    n_atom: float,
    d_atom: float,
    l_cyl: float,
    d_cyl: float,
    pbc: bool = False
) -> float:
    """
    Compute the volume fraction of a species in a cylindrical confinement.

    Parameters
    ----------
    n_atom : float
        Number of particles.
    d_atom : float
        Diameter of the particle of the species.
    l_cyl : float
        Length of the cylindrical confinement.
    d_cyl : float
        Diameter of the cylindrical confinement.
    pbc : bool
        Periodic boundary conditions along the longitudinal axis. If `True`,
        :math:`v_{avail} = \\pi * l_{cyl} * (d_{cyl} - d_{atom})^2 / 4`;
        otherwise, :math:`v_{avail} = \\pi * (l_cyl - d_{atom}) * (d_{cyl}
        - d_{atom})^2 / 4`. Defaults to `False`.

    Returns
    -------
    float
        Volume fraction of the species in the cylindrical confinement.

    Notes
    -----
    The volume fraction is computed in the volume available to the center of
    geometry of each particle. For point-like particles, the notion of volume
    fraction is meaningless. For finite-size particles, the available volume
    depends on whether periodic boundary conditions (PBCs) are applied.

    Examples
    --------
    >>> volume_fraction_cylinder(1000, 1.0, 10.0, 5.0)
    7.123792218314786
    """
    rho = number_density_cylinder(n_atom, d_atom, l_cyl, d_cyl, pbc)
    return rho * np.pi * d_atom ** 3 / 6


def spherical_segment(r: float, a: float, b: float) -> float:
    """
    Compute the volume of a spherical segment defined by two parallel planes.

    Parameters
    ----------
    r : float
        Radius of the sphere. Must be positive.
    a : float
        Distance of the first plane from the center of the sphere, along the
        axis of symmetry.
    b : float
        Distance of the second plane from the center of the sphere, along the
        axis of symmetry.

    Returns
    -------
    vol : float
        Volume of the spherical segment.

    Raises
    ------
    ValueError
        If `r` is not a positive number.

    Notes
    -----
    - `a` and `b` can be positive or negative values, as long as they fall
    within the range `[-r, r]`.
    - The function will adjust `a` and `b` to `-r` or `r` if they exceed these
    bounds.
    - If `a = r` or `b = r`, the spherical segment becomes a spherical cap.
    - If both `a` and `b` lie outside the range `[-r, r]` and share the same
    sign, the volume is zero.

    References
    ----------
    .. [1] Weisstein, Eric W. "Spherical Segment." From MathWorld--A Wolfram
    Web Resource.
       https://mathworld.wolfram.com/SphericalSegment.html

    Examples
    --------
    >>> spherical_segment(3, 1, 2)
    37.69911184307752
    >>> spherical_segment(3, -3, 3)
    113.09733552923255
    """

    if r <= 0:
        raise ValueError(f"The radius 'r' must be positive. Got {r}.")

    # Ensure the bounds are within [-r, r]
    lower = max(min(a, b), -r)
    upper = min(max(a, b), r)

    # If both bounds are outside [-r, r] with the same sign, the volume is zero
    if lower * upper >= r**2:
        return 0.0
    # Calculate the volume of the spherical segment
    vol = np.pi * (r**2 * (upper - lower) - (upper**3 - lower**3) / 3)
    return vol


def sphere_sphere_intersection(r1: float, r2: float, d: float) -> float:
    """
    Compute the volume of intersection between two spheres.

    The sphere with radius `r1` is separated from the sphere with radius `r2`
    by a distance `d` along the x-axis. Thus, the vector form of the distance
    between their centers is `(d, 0, 0)` in Cartesian coordinates.

    Parameters
    ----------
    r1 : float
        Radius of the first sphere.
    r2 : float
        Radius of the second sphere.
    d : float
        Distance between the centers of the two spheres along the x-axis.

    Returns
    -------
    vol : float
        Volume of the intersection between the two spheres.

    References
    ----------
    .. [1] Weisstein, Eric W. "Sphere-Sphere Intersection."
       From MathWorld--A Wolfram Web Resource.
       https://mathworld.wolfram.com/Sphere-SphereIntersection.html

    Examples
    --------
    >>> sphere_sphere_intersection(3, 4, 2)
    75.39822368615503
    >>> sphere_sphere_intersection(3, 4, 10)
    0.0
    """

    r_max = max(r1, r2)
    r_min = min(r1, r2)

    # Volume is zero if one sphere has a radius of zero or no intersection
    # occurs:
    if r1 == 0.0 or r2 == 0.0:
        return 0.0
    if d >= r_min + r_max:
        return 0.0
    if d <= r_max - r_min:
        # The smaller sphere is entirely contained within the larger one
        return 4 * np.pi * r_min**3 / 3

    # Calculate the intersection volume for overlapping spheres
    vol = (np.pi / (12 * d)) * (r_max + r_min - d)**2 * (
        d**2 + 2 * d * (r_max + r_min)
        - 3 * (r_min**2 + r_max**2)
        + 6 * r_min * r_max
    )

    return vol


def radial_cyl_histogram(
    positions: np.ndarray,
    edges: np.ndarray,
    bin_range,
    dim: int
) -> np.ndarray:
    """
    Compute the histogram of radial distances from the longitudinal axis along
    `dim` in the cylindrical coordinate system.

    Parameters
    ----------
    positions : np.ndarray
        Array of shape (n_atoms, n_dim) containing atom positions, and sorted
        by atom number form 1 to N.
    edges : np.ndarray
        A monotonically increasing array of bin edges, including the rightmost
        edge.
    bin_range : Tuple[float, float]
        The lower and upper ranges of the bins.
    dim : :py:data:`AxisT`
        Axis direction (0=x, 1=y, 2=z); see :mod:`polyphys.manage.types`.

    Returns
    -------
    hist: np.ndarray
        Histogram data

    Raises
    ------
    ValueError
        If `dim` is not one of {0, 1, 2}.
    ValueError
        If the positions array is not two-dimensional.

    Examples
    --------
    >>> import numpy as np
    >>> positions = np.array([[1, 0, 0], [0, 2, 0], [0, 0, 3]])
    >>> edges = np.array([0, 1, 2, 3])
    >>> radial_cyl_histogram(positions, edges, (0, 3), dim=2)
    array([1, 1, 0])
    """
    if dim not in {0, 1, 2}:
        raise ValueError("'dim' must be one of {0, 1, 2}.")
    if positions.ndim != 2:
        raise ValueError(
            "'positions' must be a 2D array with shape (n_atoms, n_dim).")

    trans_axes = np.roll(np.arange(3), -dim)[1:]  # selecting transverse axes
    trans_distances = np.linalg.norm(positions[:, trans_axes], axis=1)
    hist, _ = np.histogram(trans_distances, bins=edges, range=bin_range)
    return hist

In [ ]:
positions = np.array([
        [1, 0, 0],  # In (y,z) plane for dim=0 → radius = 0.0
        [0, 2, 0],  # In (x,z) plane for dim=1 → radius = 0.0
        [0, 0, 3],  # In (x,y) plane for dim=2 → radius = 0.0
        [0, 3, 4]   # dim=2 → radius = sqrt(0^2 + 3^2) = 3
    ])
edges = np.array([0, 1, 2, 3, 4])
bin_range = (0, 4)

# Test dim=2 → use x and y
hist = radial_cyl_histogram(positions, edges, bin_range, dim=1)
hist